# Pro tip: if you run the import once you get a lot of warning, but then you just run it again then the warning is gone

In [18]:
import impnb
import herMLE
import Data_processing as dp;
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import Import_EEG as IE;
from scipy import stats

img_path='/cndd/hop006/Processing/PRL_imgs'
behavior_log_path='/cndd/hop006/Github/RewardReversalPorject/BehaviorLog/'

# Part Zero: Gathering Data - every rat takes about 20s to process
#### Note: Only doing from rat_26 and up, some are not 20 days apart
#### Common electrodes: 
 Ag Disk (frontal center)',
 'L Au1 EEG',
 'L Cerebellum EEG',
 'L NAc LFP',
 'L lOFC LFP',
 'R ACC LFP',
 'R Au1 LFP',
 'R Caudate LFP',
 'R lOFC EEG'}

In [9]:
rat_2_anal=sorted(list(set(dp.day1['subject'])))[1:] # droped 25 cuz it only had one recording
days=2
behavior_log={}
for myrat in rat_2_anal:
    val_chan,chan_name,dates,rat_nums,reward_tone=IE.get_excel_info(myrat)
    rat_rlogs=[]
    rat_alogs=[]
    for day in range(days):
        rat_df=IE.get_ephys(dates[day],rat_nums[day])
#         tvec,signal,low_tone,high_tone,tar,off_tar=IE.rat_summary(rat_df)
        action_log,reward_log,epoched_eeg=IE.get_behavior(rat_df,eeg_length=300,plot=False)
        rat_rlogs.append(reward_log)
        rat_alogs.append(action_log)
    behavior_log[myrat]={}
    behavior_log[myrat]['reward']=rat_rlogs
    behavior_log[myrat]['action']=rat_alogs

/home/AD/hop006/anaconda3/lib/python3.7/site-packages/pandas/util/_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
Import EEG.ipynb:12: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  "import numpy as np\n",


In [20]:
behavior_df=pd.DataFrame(behavior_log)
behavior_df.to_pickle(behavior_log_path+'rat26to38.pkl')

# Part One: Behavior Models
### Model-1: Q-learning with alpha_gain and loss, beta, and sigmoid action decision

In [6]:
alphaGs,alphaLs,betas=[],[],[]
for rlog,alog in zip(rat_rlogs,rat_alogs): #
    alphaG,alphaL,beta=herMLE.my_minimize(alog,rlbog)
    print(alphaG,alphaL,beta)
    alphaGs.append(alphaG)
    alphaLs.append(alphaL)
    betas.append(beta)

0.05 0.9500000000000001 0.5
0.9500000000000001 0.9500000000000001 1.0


### Plots

# Part two: import EEG